<a href="https://colab.research.google.com/github/paoloartasensi/sound_maker/blob/main/SOUND_MAKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### START FROM HERE

In [29]:
!pip install pydub

# 3 SECONDS COUNTDOWN



In [45]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio
from pydub import AudioSegment
from io import BytesIO

def note_to_frequency(note):
    # Define the reference frequency for A4 (440 Hz)
    ref_freq = 440
    # Define the note-to-semitone mapping
    note_map = {"C": -9, "C#": -8, "D": -7, "D#": -6, "E": -5, "F": -4, "F#": -3, "G": -2, "G#": -1, "A": 0, "A#": 1, "B": 2}
    # Extract the note name and octave from the input string
    note_name = note[:-1]
    octave = int(note[-1])
    # Calculate the number of semitones away from A4
    semitones = note_map[note_name] + (octave - 4) * 12
    # Calculate the frequency using the formula: frequency = ref_freq * 2^(semitones/12)
    frequency = ref_freq * (2 ** (semitones / 12))
    return frequency

# Set the parameters for the beep sounds
beep_duration = 0.1  # duration of each beep in seconds
beep_note = "G6"  # desired note for the regular beeps
long_beep_duration = 1
long_beep_note = "C7"
sample_rate = 44100  # sample rate in Hz
decibel_reduction = -20  # volume reduction in decibels (adjust this value to change the volume)

# Convert the notes to frequencies
beep_frequency = note_to_frequency(beep_note)
long_beep_frequency = note_to_frequency(long_beep_note)

# Generate the time arrays for the beeps
beep_t = np.linspace(0, beep_duration, int(sample_rate * beep_duration), endpoint=False)
long_beep_t = np.linspace(0, long_beep_duration, int(sample_rate * long_beep_duration), endpoint=False)

# Generate the sine waves for the beeps
beep = np.sin(2 * np.pi * beep_frequency * beep_t)
long_beep = np.sin(2 * np.pi * long_beep_frequency * long_beep_t)

# Normalize the amplitudes to the range [-1, 1]
beep *= 0.5
long_beep *= 0.5

# Create the countdown sequence
countdown = np.concatenate((beep, np.zeros(int(sample_rate * 1)),  # 1-second silence
                            beep, np.zeros(int(sample_rate * 1)),  # 1-second silence
                            beep, np.zeros(int(sample_rate * 1)),  # 1-second silence
                            long_beep))

# Scale the audio data to the range [-32768, 32767]
countdown = np.int16(countdown * 32767)

# Create an AudioSegment object from the scaled audio data
audio_segment = AudioSegment(
    countdown.tobytes(),
    frame_rate=sample_rate,
    sample_width=countdown.dtype.itemsize,
    channels=1
)

# Reduce the volume of the audio by the specified number of decibels
reduced_volume_audio_segment = audio_segment.apply_gain(decibel_reduction)

# Convert the AudioSegment object to an IPython Audio object
buffer = BytesIO()
reduced_volume_audio_segment.export(buffer, format="wav")
reduced_volume_audio = Audio(buffer.getvalue(), rate=sample_rate, autoplay=True)

# Play the countdown sequence with reduced volume
display(reduced_volume_audio)

## WRONG SOUND

In [ ]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio

# Set the parameters for the "wrong" sound
duration = 1.0  # duration of the sound in seconds
frequencies = [440, 466.16, 220, 233.08]  # frequencies for the discordant sound
amplitudes = [0.5, 0.5, 0.5, 0.5]  # amplitudes for each frequency
sample_rate = 44100  # sample rate in Hz

# Generate the time array
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Generate the discordant sound
wrong_sound = np.zeros_like(t)
for freq, amp in zip(frequencies, amplitudes):
    wrong_sound += amp * np.sin(2 * np.pi * freq * t)

# Normalize the amplitude to the range [-1, 1]
wrong_sound /= np.max(np.abs(wrong_sound))

# Apply an envelope to the sound for a smoother start and end
envelope = np.linspace(0, 1, int(sample_rate * 0.05))  # 50ms attack
envelope = np.concatenate((envelope, np.ones(int(sample_rate * (duration - 0.1)))))  # sustain
envelope = np.concatenate((envelope, np.linspace(1, 0, int(sample_rate * 0.05))))  # 50ms release
wrong_sound *= envelope

# Play the "wrong" sound
Audio(wrong_sound, rate=sample_rate, autoplay=True)

# TRIGGER SOUND

DURATION OF THE SOUND CAN BE SYNC WITH THE SAMPLE AMPLITUDE, CHANGING THE START and END FREQ TOO

In [ ]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio
from pydub import AudioSegment
from io import BytesIO

# Set the parameters for the "threshold surpassed" sound
duration = 0.5  # DURATION OF THE SOUND CAN BE SYNC WITH THE SAMPLE AMPLITUDE
start_freq = 100  # starting frequency in Hz
end_freq = 300  # ending frequency in Hz
max_amplitude = 0.8  # maximum amplitude of the sound
sample_rate = 44100  # sample rate in Hz
decibel_reduction = -20  # volume reduction in decibels (adjust this value to change the volume)

# Generate the time array
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Generate the frequency array with exponential growth
freq_array = np.logspace(np.log10(start_freq), np.log10(end_freq), len(t))

# Generate the amplitude array with linear growth
amp_array = np.linspace(0, max_amplitude, len(t))

# Generate the "threshold surpassed" sound
threshold_sound = amp_array * np.sin(2 * np.pi * freq_array * t)

# Apply an envelope to the sound for a smoother start and end
envelope = np.linspace(0, 1, int(sample_rate * 0.1))  # 100ms attack
envelope = np.concatenate((envelope, np.ones(int(sample_rate * (duration - 0.2)))))  # sustain
envelope = np.concatenate((envelope, np.linspace(1, 0, int(sample_rate * 0.1))))  # 100ms release
threshold_sound *= envelope

# Scale the audio data to the range [-32768, 32767]
threshold_sound = np.int16(threshold_sound * 32767)

# Create an AudioSegment object from the scaled audio data
audio_segment = AudioSegment(
    threshold_sound.tobytes(),
    frame_rate=sample_rate,
    sample_width=threshold_sound.dtype.itemsize,
    channels=1
)

# Reduce the volume of the audio by the specified number of decibels
reduced_volume_audio_segment = audio_segment.apply_gain(decibel_reduction)

# Convert the AudioSegment object to an IPython Audio object
buffer = BytesIO()
reduced_volume_audio_segment.export(buffer, format="wav")
reduced_volume_audio = Audio(buffer.getvalue(), rate=sample_rate, autoplay=True)

# Play the "threshold surpassed" sound with reduced volume
display(reduced_volume_audio)

# SINGLE **BEEP** G6, 0.2

In [ ]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio

def note_to_frequency(note):
    # Define the reference frequency for A4 (440 Hz)
    ref_freq = 440
    # Define the note-to-semitone mapping
    note_map = {"C": -9, "C#": -8, "D": -7, "D#": -6, "E": -5, "F": -4, "F#": -3, "G": -2, "G#": -1, "A": 0, "A#": 1, "B": 2}
    # Extract the note name and octave from the input string
    note_name = note[:-1]
    octave = int(note[-1])
    # Calculate the number of semitones away from A4
    semitones = note_map[note_name] + (octave - 4) * 12
    # Calculate the frequency using the formula: frequency = ref_freq * 2^(semitones/12)
    frequency = ref_freq * (2 ** (semitones / 12))
    return frequency

# Set the parameters for the "surpassed" sound
duration = 0.2  # duration of the sound in seconds
note = "G6"  # desired note for the sound (e.g., "A4", "C#5", "F3")
amplitude = 0.8  # amplitude of the sound
sample_rate = 44100  # sample rate in Hz

# Convert the note to frequency
freq = note_to_frequency(note)

# Generate the time array
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Generate the "surpassed" sound
surpassed_sound = amplitude * np.sin(2 * np.pi * freq * t)

# Apply an envelope to the sound for a smoother start and end
envelope = np.linspace(0, 1, int(sample_rate * 0.02))  # 20ms attack
envelope = np.concatenate((envelope, np.ones(int(sample_rate * (duration - 0.04)))))  # sustain
envelope = np.concatenate((envelope, np.linspace(1, 0, int(sample_rate * 0.02))))  # 20ms release
surpassed_sound *= envelope

# Play the "surpassed" sound
Audio(surpassed_sound, rate=sample_rate, autoplay=True)

# LONG **BEEP** C7, 0.8

In [ ]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio

def note_to_frequency(note):
    # Define the reference frequency for A4 (440 Hz)
    ref_freq = 440
    # Define the note-to-semitone mapping
    note_map = {"C": -9, "C#": -8, "D": -7, "D#": -6, "E": -5, "F": -4, "F#": -3, "G": -2, "G#": -1, "A": 0, "A#": 1, "B": 2}
    # Extract the note name and octave from the input string
    note_name = note[:-1]
    octave = int(note[-1])
    # Calculate the number of semitones away from A4
    semitones = note_map[note_name] + (octave - 4) * 12
    # Calculate the frequency using the formula: frequency = ref_freq * 2^(semitones/12)
    frequency = ref_freq * (2 ** (semitones / 12))
    return frequency

# Set the parameters for the "surpassed" sound
duration = 0.8  # duration of the sound in seconds
note = "C7"  # desired note for the sound (e.g., "A4", "C#5", "F3")
amplitude = 0.8  # amplitude of the sound
sample_rate = 44100  # sample rate in Hz

# Convert the note to frequency
freq = note_to_frequency(note)

# Generate the time array
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Generate the "surpassed" sound
surpassed_sound = amplitude * np.sin(2 * np.pi * freq * t)

# Apply an envelope to the sound for a smoother start and end
envelope = np.linspace(0, 1, int(sample_rate * 0.02))  # 20ms attack
envelope = np.concatenate((envelope, np.ones(int(sample_rate * (duration - 0.04)))))  # sustain
envelope = np.concatenate((envelope, np.linspace(1, 0, int(sample_rate * 0.02))))  # 20ms release
surpassed_sound *= envelope

# Play the "surpassed" sound
Audio(surpassed_sound, rate=sample_rate, autoplay=True)

# THE SWITCH E7 is the classic WAKEUP TIMER REPETER

In [35]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio
from pydub import AudioSegment
from io import BytesIO

def note_to_frequency(note):
    # Define the reference frequency for A4 (440 Hz)
    ref_freq = 440
    # Define the note-to-semitone mapping
    note_map = {"C": -9, "C#": -8, "D": -7, "D#": -6, "E": -5, "F": -4, "F#": -3, "G": -2, "G#": -1, "A": 0, "A#": 1, "B": 2}
    # Extract the note name and octave from the input string
    note_name = note[:-1]
    octave = int(note[-1])
    # Calculate the number of semitones away from A4
    semitones = note_map[note_name] + (octave - 4) * 12
    # Calculate the frequency using the formula: frequency = ref_freq * 2^(semitones/12)
    frequency = ref_freq * (2 ** (semitones / 12))
    return frequency

# Set the parameters for the sound
duration = 0.08  # duration of the sound in seconds
note = "E6"  # desired note for the sound (e.g., "A4", "C#5", "F3")
amplitude = 0.3  # amplitude of the sound
sample_rate = 44100  # sample rate in Hz
num_repetitions = 3  # number of times to repeat the sound
distortion_amount = 1  # amount of distortion to apply
decibel_reduction = -20  # volume reduction in decibels (adjust this value to change the volume)

# Convert the note to frequency
freq = note_to_frequency(note)

# Generate the time array
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Generate the sound
surpassed_sound = amplitude * np.sin(2 * np.pi * freq * t)

# Apply distortion to the sound
distorted_sound = np.tanh(distortion_amount * surpassed_sound) / distortion_amount

# Apply an envelope to the distorted sound for a smoother start and end
envelope = np.linspace(0, 1, int(sample_rate * 0.02))  # 20ms attack
envelope = np.concatenate((envelope, np.ones(int(sample_rate * (duration - 0.04)))))  # sustain
envelope = np.concatenate((envelope, np.linspace(1, 0, int(sample_rate * 0.02))))  # 20ms release
distorted_sound *= envelope

# Repeat the distorted sound for the specified number of repetitions
repeated_sound = np.tile(distorted_sound, num_repetitions)

# Scale the audio data to the range [-32768, 32767]
repeated_sound = np.int16(repeated_sound * 32767)

# Create an AudioSegment object from the scaled audio data
audio_segment = AudioSegment(
    repeated_sound.tobytes(),
    frame_rate=sample_rate,
    sample_width=repeated_sound.dtype.itemsize,
    channels=1
)

# Reduce the volume of the audio by the specified number of decibels
reduced_volume_audio_segment = audio_segment.apply_gain(decibel_reduction)

# Convert the AudioSegment object to an IPython Audio object
buffer = BytesIO()
reduced_volume_audio_segment.export(buffer, format="wav")
reduced_volume_audio = Audio(buffer.getvalue(), rate=sample_rate, autoplay=True)

# Play the repeated distorted sound with reduced volume
display(reduced_volume_audio)

CAR BELT SECURE SOUND

In [40]:
import numpy as np
from IPython.display import Audio
from pydub import AudioSegment
from io import BytesIO

# Define the audio parameters
duration = 1.5  # seconds
sampling_freq = 44100  # Hz
amplitude = 0.5  # between 0 and 1
decibel_reduction = -20  # volume reduction in decibels (adjust this value to change the volume)

# Generate time points
time_points = np.linspace(0, duration, int(sampling_freq * duration), endpoint=False)

# Generate the bell sound waveform
freq_bell = 500  # Hz
bell_sound = amplitude * np.sin(2 * np.pi * freq_bell * time_points)

# Apply an exponential decay to simulate the bell's decay
decay_rate = 5  # adjust this to change the decay speed
bell_sound *= np.exp(-decay_rate * time_points)

# Scale the waveform to 16-bit integer range
audio = np.int16(bell_sound * 32767)

# Create an AudioSegment object from the scaled audio data
audio_segment = AudioSegment(
    audio.tobytes(),
    frame_rate=sampling_freq,
    sample_width=audio.dtype.itemsize,
    channels=1
)

# Reduce the volume of the audio by the specified number of decibels
reduced_volume_audio_segment = audio_segment.apply_gain(decibel_reduction)

# Convert the AudioSegment object to an IPython Audio object
buffer = BytesIO()
reduced_volume_audio_segment.export(buffer, format="wav")
reduced_volume_audio = Audio(buffer.getvalue(), rate=sampling_freq, autoplay=True)

# Play the audio in Colab with reduced volume
display(reduced_volume_audio)